<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=07e7178ea6b8b5bbc42e20c68fa41446335474920f51f133d26d30da0e934f48
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-17 16:04:07
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.40 C
-------------------
Today PnL: -15.96 K (-0.11%)
Current PnL: -21.24 L (-14.15%)
CY Booked + Current PnL: -9.34 L (-6.22%)
-------------------
Total profit:  1.81 L
Total loss:  -23.05 L
-------------------
Total Booked + Current PnL: 17.64 L (14.33%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.49%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.40 L (63.03%)
Deployed:  1.23 C
Current:  1.40 C
CAGR/XIRR %: 7.73%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-19.09,61.0,X-LC,3.89,216131.0,11631.0,11671.0,0.52,5.69,5.40,11.39,10.0,1.00,1.55,15.15,X40N,ATH,PHARMA
83,VOLTAS,1530.00,2.00,60.0,X-MC,1.34,213990.0,22248.0,15514.0,0.78,11.60,7.25,19.69,99.0,1.43,1.54,19.19,XY25,NTT,AC
76,TTKPRESTIG,770.00,95.30,47.0,M-SC,1.69,84128.0,-16649.0,16741.0,-1.82,-16.52,19.90,0.09,245.0,-0.99,0.60,10.53,OX40N,NTT,DURABLES
56,RELIANCE,1533.00,-11.79,63.0,X-LC,2.85,221021.0,10655.0,18124.0,1.32,5.06,8.20,13.68,37.0,0.59,1.59,22.42,XY25,NTT,REFINERIES
33,ICICIGI,2260.25,-13.84,64.0,X-MC,2.32,148074.0,11941.0,19190.0,-0.45,8.77,12.96,22.86,91.0,0.62,1.06,22.49,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-38.52,69.0,M-SC,4.16,103830.0,12332.0,66420.0,11.84,13.48,63.97,86.07,223.0,0.19,0.75,50.78,XR,NTT,DURABLES
25,GREENPANEL,537.00,239.15,59.0,M-SC,4.23,151378.0,-31700.0,110672.0,4.29,-17.32,73.11,43.14,230.0,-0.29,1.09,42.39,XY24,NTT,MISC
5,ASIANPAINT,3465.66,-12.54,65.0,X-LC,5.30,220686.0,-31082.0,84302.0,4.07,-12.35,38.20,21.13,27.0,-0.37,1.59,18.71,X40,ATH,PAINTS
45,KPIGREEN,731.05,1.34,58.0,H-SC,6.15,120569.0,-4728.0,63660.0,4.07,-3.77,52.80,47.03,141.0,-0.07,0.87,50.96,MH,ATH,POWER
44,KANSAINER,340.00,-66.87,61.0,H-SC,1.90,227934.0,-41733.0,78067.0,3.30,-15.48,34.25,13.47,138.0,-0.53,1.64,15.90,XY24,NTT,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-15.15,43.0,M-LC,5.72,149358.0,-1587.0,111048.0,-5.09,-1.05,74.35,72.51,53.0,-0.01,1.07,4.91,XR,NTT,IT
70,TANLA,1963.11,-23.20,44.0,H-SC,19.65,206412.0,-64349.0,394309.0,-4.68,-23.77,191.03,121.86,133.0,-0.16,1.48,61.47,AR,ATH,IT
63,SHALBY,327.00,1155.18,51.0,M-SC,23.28,170651.0,-10492.0,55956.0,-2.57,-5.79,32.79,25.10,235.0,-0.19,1.23,36.76,XY24,NTT,HEALTHCARE
7,ATULAUTO,844.00,3624.32,49.0,M-SC,3.74,167144.0,-29988.0,119809.0,-2.31,-15.21,71.68,45.57,236.0,-0.25,1.20,18.50,XY24,NTT,AUTO
50,PGHH,17907.65,-28.69,46.0,X-MC,3.32,206370.0,5550.0,62241.0,-2.31,2.76,30.16,33.76,80.0,0.09,1.48,8.10,X40,ATH,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,69.0,M-SC,18.7,150084.0,864.0,143315.0,-0.08,0.58,95.49,96.62,208.0,0.01,1.08,62.42,XY25,NTT,FINANCE
36,INDIAMART,4810.62,-55.57,39.0,H-SC,7.9,124237.0,901.0,130722.0,-0.51,0.73,105.22,106.72,119.0,0.01,0.89,24.28,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,107.70,60.0,H-SC,12.33,131544.0,-10935.0,30452.0,-0.55,-7.67,23.15,13.70,163.0,-0.36,0.95,55.26,OX40N,NTT,BANKS
44,KANSAINER,340.0,-66.87,61.0,H-SC,1.90,227934.0,-41733.0,78067.0,3.30,-15.48,34.25,13.47,138.0,-0.53,1.64,15.90,XY24,NTT,PAINTS
76,TTKPRESTIG,770.0,95.30,47.0,M-SC,1.69,84128.0,-16649.0,16741.0,-1.82,-16.52,19.90,0.09,245.0,-0.99,0.60,10.53,OX40N,NTT,DURABLES
17,CERA,9475.0,-24.34,38.0,H-SC,2.36,138046.0,-37857.0,79873.0,-0.61,-21.52,57.86,23.89,149.0,-0.47,0.99,19.22,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.55,44.0,H-MC,7.34,102020.0,-29815.0,72036.0,-0.63,-22.62,70.61,32.03,98.0,-0.41,0.73,14.27,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,45.36,49.0,M-LC,6.22,206222.0,3052.0,39079.0,-0.68,1.50,18.95,20.74,55.0,0.08,1.48,7.13,XY25,NTT,FINANCE
31,HINDZINC,730.22,33.63,61.0,M-LC,9.97,217609.0,12533.0,100035.0,-1.27,6.11,45.97,54.89,52.0,0.13,1.56,31.68,X5K,ATH,METALS
78,UNIONBANK,163.00,-14.73,50.0,M-LC,8.74,155417.0,14577.0,29918.0,-1.53,10.35,19.25,31.59,66.0,0.49,1.12,39.58,XY24,NTT,BANKS
11,BANKINDIA,190.00,-26.71,53.0,H-MC,12.16,189343.0,9535.0,102870.0,-1.84,5.30,54.33,62.52,88.0,0.09,1.36,41.20,XR,NTT,BANKS
84,WHIRLPOOL,2270.00,-38.52,69.0,M-SC,4.16,103830.0,12332.0,66420.0,11.84,13.48,63.97,86.07,223.0,0.19,0.75,50.78,XR,NTT,DURABLES


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-38.52,69.0,M-SC,4.16,103830.0,12332.0,66420.0,11.84,13.48,63.97,86.07,223.0,0.19,0.75,50.78,XR,NTT,DURABLES
31,HINDZINC,730.22,33.63,61.0,M-LC,9.97,217609.0,12533.0,100035.0,-1.27,6.11,45.97,54.89,52.0,0.13,1.56,31.68,X5K,ATH,METALS
11,BANKINDIA,190.00,-26.71,53.0,H-MC,12.16,189343.0,9535.0,102870.0,-1.84,5.30,54.33,62.52,88.0,0.09,1.36,41.20,XR,NTT,BANKS
36,INDIAMART,4810.62,-55.57,39.0,H-SC,7.90,124237.0,901.0,130722.0,-0.51,0.73,105.22,106.72,119.0,0.01,0.89,24.28,AR,ATH,MISC
85,WIPRO,420.00,-15.15,43.0,M-LC,5.72,149358.0,-1587.0,111048.0,-5.09,-1.05,74.35,72.51,53.0,-0.01,1.07,4.91,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.0,-54.49,14.0,X-LC,7.45,143966.0,-132557.0,242626.0,-0.05,-47.94,168.53,39.81,54.0,-0.55,1.03,1.47,XY24,NTT,AUTO
51,QUESS,986.0,-48.21,32.0,X-SC,38.69,51268.0,-13738.0,164668.0,-1.78,-21.13,321.19,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
39,INFY,2275.0,-21.03,40.0,X-LC,7.49,306954.0,-5934.0,177634.0,-2.07,-1.90,57.87,54.87,3.0,-0.03,2.21,4.63,X40,BTT,IT
12,BATAINDIA,2096.0,-36.15,42.0,X-SC,4.31,95475.0,-33195.0,78490.0,0.23,-25.80,82.21,35.20,219.0,-0.42,0.69,9.58,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.5,-14.90,42.0,X-MC,6.04,179465.0,-21041.0,118196.0,-0.55,-10.49,65.86,48.45,90.0,-0.18,1.29,3.49,X40N,ATH,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-2.83,45.0,H-LC,0.40,156060.0,-30035.0,77515.0,0.07,-16.14,49.67,25.51,15.0,-0.39,1.12,15.00,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,2.00,60.0,X-MC,1.34,213990.0,22248.0,15514.0,0.78,11.60,7.25,19.69,99.0,1.43,1.54,19.19,XY25,NTT,AC
28,HAVELLS,2069.16,-6.18,44.0,X-MC,2.09,232674.0,-15057.0,90115.0,1.06,-6.08,38.73,30.30,92.0,-0.17,1.67,6.41,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-13.84,64.0,X-MC,2.32,148074.0,11941.0,19190.0,-0.45,8.77,12.96,22.86,91.0,0.62,1.06,22.49,X40,ATH,INSURANCE
41,ITC,452.00,-37.50,59.0,X-LC,2.63,201541.0,1403.0,19489.0,1.73,0.70,9.67,10.44,4.0,0.07,1.45,6.30,X40,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,32.0,X-SC,38.69,51268.0,-13738.0,164668.0,-1.78,-21.13,321.19,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-54.49,14.0,X-LC,7.45,143966.0,-132557.0,242626.0,-0.05,-47.94,168.53,39.81,54.0,-0.55,1.03,1.47,XY24,NTT,AUTO
2,ACC,3906.00,-38.50,44.0,X-MC,3.27,183270.0,-54681.0,207333.0,-1.47,-22.98,113.13,64.15,174.0,-0.26,1.32,2.80,XY24,BTT,CEMENT
73,TCS,4389.97,-29.97,44.0,X-LC,12.65,278447.0,-67509.0,134211.0,-0.29,-19.51,48.20,19.28,1.0,-0.50,2.00,2.94,X40,ATH,IT
9,BAJAJHFL,181.50,-14.90,42.0,X-MC,6.04,179465.0,-21041.0,118196.0,-0.55,-10.49,65.86,48.45,90.0,-0.18,1.29,3.49,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,32.0,X-SC,38.69,51268.0,-13738.0,164668.0,-1.78,-21.13,321.19,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
55,RELAXO,1176.00,-39.59,43.0,X-SC,4.40,82391.0,-62769.0,138697.0,-0.67,-43.24,168.34,52.31,136.0,-0.45,0.59,10.28,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-36.15,42.0,X-SC,4.31,95475.0,-33195.0,78490.0,0.23,-25.80,82.21,35.20,219.0,-0.42,0.69,9.58,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-6.49,45.0,X-SC,0.86,106280.0,4276.0,39653.0,-0.92,4.19,37.31,43.06,122.0,0.11,0.76,22.03,X40N,ATH,MISC
32,HONAUT,58357.33,-22.75,48.0,X-SC,2.51,108930.0,-19008.0,66142.0,0.61,-14.86,60.72,36.84,143.0,-0.29,0.78,11.52,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-29.97,44.0,X-LC,12.65,278447.0,-67509.0,134211.0,-0.29,-19.51,48.20,19.28,1.0,-0.50,2.00,2.94,X40,ATH,IT
39,INFY,2275.00,-21.03,40.0,X-LC,7.49,306954.0,-5934.0,177634.0,-2.07,-1.90,57.87,54.87,3.0,-0.03,2.21,4.63,X40,BTT,IT
41,ITC,452.00,-37.50,59.0,X-LC,2.63,201541.0,1403.0,19489.0,1.73,0.70,9.67,10.44,4.0,0.07,1.45,6.30,X40,NTT,FMCG
82,VBL,671.64,-18.24,54.0,X-LC,10.52,294469.0,-21373.0,134042.0,0.05,-6.77,45.52,35.67,5.0,-0.16,2.12,6.15,X40N,ATH,FMCG
1,ABB,7934.00,-40.25,50.0,H-LC,7.08,249696.0,-11923.0,131140.0,-0.19,-4.56,52.52,45.57,7.0,-0.09,1.79,6.25,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6794.44,59.0,L-SC,6.26,75623.0,-18187.0,94809.0,0.63,-19.39,125.37,81.67,269.0,-0.19,0.54,46.45,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,29.16,81350.0,-32199.0,72247.0,1.48,-28.36,88.81,35.27,268.0,-0.45,0.58,99.11,XR,NTT,HOTELS
48,MASFIN,398.61,-21.41,39.0,H-SC,9.46,89220.0,-8760.0,30362.0,-1.80,-8.94,34.03,22.05,152.0,-0.29,0.64,29.69,XR,ATH,FINANCE
84,WHIRLPOOL,2270.00,-38.52,69.0,M-SC,4.16,103830.0,12332.0,66420.0,11.84,13.48,63.97,86.07,223.0,0.19,0.75,50.78,XR,NTT,DURABLES
77,UJJIVANSFB,60.00,107.70,60.0,H-SC,12.33,131544.0,-10935.0,30452.0,-0.55,-7.67,23.15,13.70,163.0,-0.36,0.95,55.26,OX40N,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,69.0,M-SC,18.70,150084.0,864.0,143315.0,-0.08,0.58,95.49,96.62,208.0,0.01,1.08,62.42,XY25,NTT,FINANCE
67,STARHEALTH,761.00,22.98,72.0,H-SC,9.24,274149.0,-6974.0,139843.0,2.75,-2.48,51.01,47.26,144.0,-0.05,1.97,46.88,XY24,NTT,INSURANCE
4,ANGELONE,3033.00,15.82,69.0,X-SC,8.84,204713.0,13707.0,43993.0,0.56,7.18,21.49,30.21,157.0,0.31,1.47,29.56,X40N,NTT,FINANCE
31,HINDZINC,730.22,33.63,61.0,M-LC,9.97,217609.0,12533.0,100035.0,-1.27,6.11,45.97,54.89,52.0,0.13,1.56,31.68,X5K,ATH,METALS
63,SHALBY,327.00,1155.18,51.0,M-SC,23.28,170651.0,-10492.0,55956.0,-2.57,-5.79,32.79,25.10,235.0,-0.19,1.23,36.76,XY24,NTT,HEALTHCARE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.64
1,25,44.24
2,50,75.85


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.35
LC    33.83
MC    21.81
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.58
X40      14.80
XY25     12.41
XR       10.91
X40N     10.24
AR        8.06
OX40N     7.63
X200      1.81
X5K       1.56
SR        1.12
MH        0.87
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.41
X-LC    22.85
X-MC    16.13
M-SC    12.82
M-LC     5.23
H-LC     4.72
X-SC     4.66
H-MC     3.78
M-MC     1.59
L-SC     1.46
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.30,-3.12,37.11
IT,12.70,-18.86,83.01
FINANCE,12.16,-11.72,62.36
MISC,6.88,-20.04,86.54
BANKS,6.33,-13.54,73.27
PAINTS,5.74,-15.87,33.15
ELECTRICAL,5.36,-10.17,48.83
INSURANCE,4.00,0.69,36.44
AUTO,2.81,-46.64,116.74


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3319222.0,22
XR,1391380.0,14
AR,1206363.0,9
X40,777991.0,10
X40N,601006.0,8
OX40N,586689.0,10
XY25,455907.0,8
SR,264770.0,2
X5K,100035.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3464825.0,25
M-SC,1259304.0,15
X-MC,1081761.0,11
X-LC,1058248.0,13
X-SC,531643.0,6
H-MC,354105.0,3
H-LC,281800.0,3
M-LC,280080.0,4
L-SC,257193.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1190415.0      6
           AR         835555.0      5
           XR         834596.0      7
M-SC       XY24       689132.0      6
X-MC       XY24       585424.0      4
X-LC       X40        484049.0      5
           XY24       309445.0      2
X-SC       X40N       288485.0      4
H-SC       OX40N      275829.0      4
           SR         264770.0      2
X-MC       X40        215452.0      4
H-LC       AR         208655.0      2
X-LC       X40N       194325.0      3
H-MC       XY24       179199.0      1
M-MC       XY24       171021.0      1
L-SC       XR         167056.0      2
X-SC       XY24       164668.0      1
X-MC       XY25       162689.0      2
M-SC       AR         162153.0      2
           OX40N      148687.0      4
           XY25       143315.0      1
X-MC       X40N       118196.0      1
M-SC       XR         116017.0      2
M-LC       XR         111048.0      1
H-MC       XR         102870.0      1
M-LC       X5K        100035.0      1
L-SC       OX40N       90137.0      1
X-SC       X40         78490.0      1
H-LC       X200        73145.0      1
H-MC       OX40N       72036.0      1
X-LC       XY25        70429.0      3
H-SC       MH          63660.0      1
L-MC       XR          59793.0      1
L-LC       XY25        40395.0      1
M-LC       XY25        39079.0      1
           XY24        29918.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
